# 37

In [ ]:
# 관련 라이브러리 설치
# datasets==1.4.1 버전을 설치해야 함에 유의
# !pip install datasets==1.4.1
# !pip install transformers
# !pip install soundfile
# !pip install jiwer


#38

In [3]:
import soundfile as sf
import torch
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

model_name = "elgeish/wav2vec2-base-timit-asr"

# 모델 및 프로세서 불러오기
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

# 모델을 eval 모드로 전환
model.eval()


Some weights of the model checkpoint at elgeish/wav2vec2-base-timit-asr were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at elgeish/wav2vec2-base-timit-asr and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN thi

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder)

#39

In [5]:
# 런타임 1분 30초
dataset = load_dataset("timit_asr")

ManualDownloadError:                     The dataset timit_asr with config clean requires manual data.
                    Please follow the manual download instructions:
                     To use TIMIT you have to download it manually. Please create an account and download the dataset from https://catalog.ldc.upenn.edu/LDC93S1 
Then extract all files in one folder and load the dataset with: `datasets.load_dataset('timit_asr', data_dir='path/to/folder/folder_name')`
                    Manual data can be loaded with:
                     datasets.load_dataset("timit_asr", data_dir="<path/to/manual/data>")

#40

In [ ]:
import soundfile as sf
import IPython.display as ipd
import numpy as np
import random

# dataset['test']["file"]에 있는 첫번째 데이터를 df.read( )에 투입
speech, _ = sf.read(dataset['test']["file"][0])
ipd.Audio(data=np.asarray(speech), autoplay=True, rate=16000)


#41

In [ ]:
# 음성 데이터를 프로세서로 처리
inputs = processor(speech, sampling_rate=16000, return_tensors="pt", padding="longest")

# 음성 데이터 처리 결과중에 inputs['input_values'] 차원 확인
inputs['input_values'].shape


#42

In [ ]:
# 그래디언트(기울기) 계산 방지, 이는 모델 eval 모드에 필요한 조치임
with torch.no_grad():

  # 모델에 input.input_values 투입
  outputs = model(inputs.input_values)

  # 로짓(outputs.logits)의 차원 확인
  print(outputs.logits.shape)

  # 로짓(output.logits)의 마지막 축 기준으로 argmax 적용
  # 가장 큰 값의 인덱스를 predicted_ids에 저장
  predicted_ids = outputs.logits.argmax(-1)

  # 위의 predicted_ids 출력
  print(predicted_ids)

  # predicted_ids의 첫번째 데이터를 processor.tokenizer.batch_decode)에 넣고 디코딩
  # predicted_ids[0]은 입력 문장 중 첫번째라는 뜻이나
  # 입력 문장을 하나만 넣었으므로 predicted_ids[0]은 입력 문장 그 자체를 의미함
  # 디코딩 결과를 출력
  print(f"predicted:{processor.tokenizer.batch_decode(predicted_ids)[0]}")

  # dataset['test']['text]에 있는 첫번째 문장, 즉 정답 문장 출력
  print(f"answer:{dataset['test']['text'][0]}")
